In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import openpyxl
import re
import os
from variableUtils import *
import variableUtils
from Utils import *
from ClassUtils import *
from pprint import pprint
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from reportlab.lib.pagesizes import letter, landscape, A4, A3
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak, Paragraph, Spacer, Image
from reportlab.lib import colors
from reportlab.platypus import Paragraph, Spacer, KeepTogether, KeepInFrame
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
import io
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import FormulaRule
import PIL
import win32com.client
import win32com
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(sns.__version__)


(841.68, 1190.8799999999999)
0.13.2


### Type 1

In [ ]:
class StudentReportMailer:
    def __init__(self, report_folder: str, email_file: str, col_email: str, col_id: str, col_first_name: str, col_last_name: str):
        """
        Initialize the mailer with the report folder and student email data.
        :param report_folder: Path to the folder containing student reports.
        :param email_file: Path to the file containing student email addresses.
        :param col_email: Column name containing student emails.
        :param col_id: Column name containing student IDs.
        :param col_name: Column name containing student names.
        """
        self.report_folder = report_folder
        self.col_email = col_email
        self.col_id = col_id
        self.col_first_name = col_first_name
        self.col_last_name = col_last_name
        self.email_df = pd.read_excel(email_file)  # Read the Excel file
        self.email_df.columns = self.email_df.columns.str.lower()
        # Convert necessary columns to string for matching
        self.email_df[self.col_id] = self.email_df[self.col_id].astype(str)
        self.email_df[self.col_first_name] = self.email_df[self.col_first_name].str.lower()
        self.email_df[self.col_last_name] = self.email_df[self.col_last_name].str.lower()
        self.no_email_list = []
        self.outlook = win32com.client.Dispatch("Outlook.Application")
    
    def send_emails(self):
        """
        Iterate through the report folder and send emails.
        """
        for file in os.listdir(self.report_folder):
            if file.endswith(".pdf"):
                email = self.find_email(file)
                if email:
                    self.send_email(email, file)
                    pass
                else:
                    print(f"No matching email found for: {file}")
                    self.no_email_list.append(file)
    
    def find_email(self, filename: str):
        """
        Find the email corresponding to a report filename.
        :param filename: Name of the file
        :return: Matching email or None
        """
        match = re.search(r"\((\d+)\)", filename)  # Match student ID format (ID)
        if match:
            student_id = match.group(1).strip()
            email_row = self.email_df[self.email_df[self.col_id] == student_id]
            if not email_row.empty:
                return email_row.iloc[0][self.col_email]
        else:
            # Match using name if no ID is found
            name = os.path.splitext(filename)[0].lower()  # Remove file extension
            first_name = name.split(" ")[0]
            last_name = " ".join(name.split(" ")[1:])
            print(f"First name: {first_name}, Last name: {last_name}")
            email_row = self.email_df[(self.email_df[self.col_first_name] == first_name) & (self.email_df[self.col_last_name] == last_name)]
            if not email_row.empty:
                return email_row.iloc[0][self.col_email]
        return None
    
    def send_email(self, recipient_email: str, filename: str):
        """
        Send an email with the report attachment.
        :param recipient_email: The email of the recipient.
        :param filename: The report file to attach.
        """
        mail = self.outlook.CreateItem(0)
        mail.To = recipient_email
        mail.Subject = "Student Submission Report"
        mail.Body = "Dear Student,\n\nPlease find attached your assessment submissions.\n\nBest regards,\nKunal Patel"
        filename = filename.strip()
        full_path = os.path.join(self.report_folder, filename)
        print(f"Sending email to {recipient_email} with attachment {full_path}")
        self.report_folder = os.path.abspath(self.report_folder)
        mail.Attachments.Add(os.path.join(self.report_folder, filename))
        mail.Send()
        print(f"Email sent to {recipient_email} with attachment {full_path}")

# Example usage:
mailer = StudentReportMailer(report_folder="2025\\DDS2\\Student Reports", email_file=variableUtils.studentEmailFile, 
                             col_email="email", col_id="student id", col_first_name="first name", col_last_name="last name")
# mailer.send_emails()


In [ ]:
not_send_list = ['Bella Dong.pdf', 'Caroline Twang Kwong Hong.pdf', 'Chan Huy Tu.pdf', 'Karn Chohan.pdf',
                'Michelle Onyekweli.pdf', 'Ming Wai Ashley Lui.pdf', 'Muhammad Yusuf Omran.pdf', 
                'Patrick O’Donoghue.pdf', 'Justin Qiu.pdf', 'Phoebe Angelene Del Rosario.pdf',
                 'Sheng Weng Selwyn Lo.pdf', 'Xiao Yang Wang.pdf', 'Daniel Chu.pdf'
                 ]

email_list = ['bella.dong@student.unimelb.edu.au', 'caroline.tsangkwonghong@student.unimelb.edu.au', 'chanhuy.tu@student.unimelb.edu.au', 'gurkarn.chohan@student.unimelb.edu.au', 
              'ogochukwu.onyekweli@student.unimelb.edu.au', 'mingwaiashley.lui@student.unimelb.edu.au', 'yusuf.omran1@student.unimelb.edu.au', 
              'pjodonoghue@student.unimelb.edu.au', 'ruijun.qiu@student.unimelb.edu.au', 'pdelrosario@student.unimelb.edu.au',
              'shengweng.lo@student.unimelb.edu.au', 'xiaoyang.wang@student.unimelb.edu.au', 'mingjun.chu@student.unimelb.edu.au']

dict_email = dict(zip(not_send_list, email_list))
for file, email in dict_email.items():
    mailer.send_email(email, file)
    pass

## Going through list first

In [2]:
class StudentReportMailer:
    def __init__(self, report_folder: str, cohort: str, email_file: str, col_email: str, col_id: str, col_first_name: str, col_last_name: str, col_cohort: str, name_type: int = 1):
        """
        Initialize the mailer with the report folder and student email data.
        :param report_folder: Path to the folder containing student reports.
        :param email_file: Path to the file containing student email addresses.
        :param col_email: Column name containing student emails.
        :param col_id: Column name containing student IDs.
        :param col_name: Column name containing student names.
        """
        self.report_folder = report_folder
        self.col_email = col_email
        self.col_id = col_id
        self.col_first_name = col_first_name
        self.col_last_name = col_last_name
        self.col_cohort = col_cohort
        self.cohort = cohort
        self.name_type = name_type


        self.email_df = pd.read_excel(email_file)  # Read the Excel file
        self.email_df.columns = self.email_df.columns.str.lower()
        # Convert necessary columns to string for matching
        self.email_df[self.col_id] = self.email_df[self.col_id].astype('Int64')
        self.email_df[self.col_id] = self.email_df[self.col_id].astype(str)
        # self.email_df[self.col_first_name] = self.email_df[self.col_first_name].str.lower()
        # self.email_df[self.col_last_name] = self.email_df[self.col_last_name].str.lower()
        self.no_email_list = []
        self.outlook = win32com.client.Dispatch("Outlook.Application")
    
    def send_emails(self):
        """
        Iterate through the report folder and send emails.
        """
        self.cohort_df = self.email_df[self.email_df[self.col_cohort] == self.cohort]
        print(f"Sending emails for cohort: {self.cohort}, with size: {self.cohort_df.shape}")
        for i, row in self.cohort_df.iterrows():

            student_id = row[self.col_id]
            student_first_name = row[self.col_first_name]
            student_last_name = row[self.col_last_name]
            student_email = row[self.col_email]
            print(f"Sending email to {student_email} for {student_first_name} {student_last_name} ({student_id}), type: {self.name_type}")
            if self.name_type == 1:
                student_name = f"{student_last_name} {student_first_name} ({student_id})"
                report_file = [file for file in os.listdir(self.report_folder) if student_id in file]
                if len(report_file) == 1:
                    report_file = report_file[0]
                    print(f"Found report file for {student_name}: {report_file}")
                elif len(report_file) > 1:
                    print(f"Multiple files found for {student_name}: {report_file}")
                    report_file = report_file[0]
                elif len(report_file) == 0:
                    print(f"No files found for {student_name}")
                    self.no_email_list.append(student_name)
                    continue
            else:
                student_name = f"{student_first_name} {student_last_name}"
                report_file = f"{student_name}.pdf"
                if report_file not in os.listdir(self.report_folder):
                    print(f"Report file not found for {student_name}")
                    self.no_email_list.append(student_name)
                    continue
                    
            self.send_email(student_email, report_file)
    
   
    def send_email(self, recipient_email: str, filename: str):
        """
        Send an email with the report attachment.
        :param recipient_email: The email of the recipient.
        :param filename: The report file to attach.
        """
        mail = self.outlook.CreateItem(0)
        mail.To = recipient_email
        mail.Subject = "Student Submission Report"
        mail.Body = "Dear Student,\n\nPlease find attached your assessment submissions.\n\nBest regards,\nKunal Patel"
        filename = filename.strip()
        full_path = os.path.join(self.report_folder, filename)
        print(f"Sending email to {recipient_email} with attachment {full_path}")
        self.report_folder = os.path.abspath(self.report_folder)
        mail.Attachments.Add(os.path.join(self.report_folder, filename))
        mail.Send()
        print(f"Email sent to {recipient_email} with attachment {full_path}\n")

# Example usage:
mailerDDS2 = StudentReportMailer(report_folder="2025\\DDS2\\Student Reports", email_file=variableUtils.studentEmailFile, 
                             col_email="email", col_id="student id", col_first_name="first name", col_last_name="last name", cohort="DDS2", col_cohort="cohort", name_type=1)

mailerDDS3GPPerio = StudentReportMailer(report_folder="2025\\DDS3\\GP+Perio\\Student Reports GP+Perio\\DDS3", email_file=variableUtils.studentEmailFile,
                             col_email="email", col_id="student id", col_first_name="first name", col_last_name="last name", cohort="DDS3", col_cohort="cohort")

mailerDDS3Specialist = StudentReportMailer(report_folder="2025\\DDS3\\Specialist\\Student Reports\\DDS3 All", email_file=variableUtils.studentEmailFile,
                             col_email="email", col_id="student id", col_first_name="first name", col_last_name="last name", cohort="DDS3", col_cohort="cohort")

mailerBOH2 = StudentReportMailer(report_folder="2025\\BOH2\\Form 1\\Student Reports", email_file=variableUtils.studentEmailFile,
                                col_email="email", col_id="student id", col_first_name="first name", col_last_name="last name", cohort="BOH2", col_cohort="cohort", name_type=1)

mailerBOH3 = StudentReportMailer(report_folder="2025\BOH3+DDS4\Session 5\Student Reports till date\BOH3", email_file=variableUtils.studentEmailFile,
                                col_email="email", col_id="student id", col_first_name="first name", col_last_name="last name", cohort="BOH3", col_cohort="cohort")

mailerDDS4 = StudentReportMailer(report_folder="2025\BOH3+DDS4\Session 5\Student Reports till date\DDS4", email_file=variableUtils.studentEmailFile,
                                col_email="email", col_id="student id", col_first_name="first name", col_last_name="last name", cohort="DDS4", col_cohort="cohort")

mailerTest = StudentReportMailer(report_folder="2025\Test", email_file=variableUtils.studentEmailFile,
                                col_email="email", col_id="student id", col_first_name="first name", col_last_name="last name", cohort="Test", col_cohort="cohort")


mailerTest.send_emails()





Sending emails for cohort: Test, with size: (1, 6)
Sending email to kunal.patel0999@gmail.com for Kunal Patel (1291822), type: 1
Found report file for Patel Kunal (1291822): Patel Kunal (1291822).pdf
Sending email to kunal.patel0999@gmail.com with attachment 2025\Test\Patel Kunal (1291822).pdf
Email sent to kunal.patel0999@gmail.com with attachment 2025\Test\Patel Kunal (1291822).pdf



In [3]:
mailerBOH3.send_emails()
print(mailerBOH3.no_email_list)


Sending emails for cohort: BOH3, with size: (54, 6)
Sending email to arzi.aktag@student.unimelb.edu.au for Arzi Aktag (1473169), type: 1
Found report file for Aktag Arzi (1473169): Aktag Arzi (1473169).pdf
Sending email to arzi.aktag@student.unimelb.edu.au with attachment 2025\BOH3+DDS4\Session 5\Student Reports till date\BOH3\Aktag Arzi (1473169).pdf
Email sent to arzi.aktag@student.unimelb.edu.au with attachment 2025\BOH3+DDS4\Session 5\Student Reports till date\BOH3\Aktag Arzi (1473169).pdf

Sending email to rami.almaalouf@student.unimelb.edu.au for Rami Al Maalouf (1337652), type: 1
Found report file for Al Maalouf Rami (1337652): Al Maalouf Rami (1337652).pdf
Sending email to rami.almaalouf@student.unimelb.edu.au with attachment c:\Users\Kunal Patel\D folder\MDS Work\2025\BOH3+DDS4\Session 5\Student Reports till date\BOH3\Al Maalouf Rami (1337652).pdf
Email sent to rami.almaalouf@student.unimelb.edu.au with attachment c:\Users\Kunal Patel\D folder\MDS Work\2025\BOH3+DDS4\Session 5

In [4]:
mailerDDS4.send_emails()
print(mailerDDS4.no_email_list)

Sending emails for cohort: DDS4, with size: (91, 6)
Sending email to ruwanyathanumi.atapattu@student.unimelb.edu.au for Ruwanya Atapattu (1328020), type: 1
Found report file for Atapattu Ruwanya (1328020): Atapattu Ruwanya (1328020).pdf
Sending email to ruwanyathanumi.atapattu@student.unimelb.edu.au with attachment 2025\BOH3+DDS4\Session 5\Student Reports till date\DDS4\Atapattu Ruwanya (1328020).pdf
Email sent to ruwanyathanumi.atapattu@student.unimelb.edu.au with attachment 2025\BOH3+DDS4\Session 5\Student Reports till date\DDS4\Atapattu Ruwanya (1328020).pdf

Sending email to atwaln@student.unimelb.edu.au for Nimret Atwal (608989), type: 1
Found report file for Atwal Nimret (608989): Atwal Nimret (608989).pdf
Sending email to atwaln@student.unimelb.edu.au with attachment c:\Users\Kunal Patel\D folder\MDS Work\2025\BOH3+DDS4\Session 5\Student Reports till date\DDS4\Atwal Nimret (608989).pdf
Email sent to atwaln@student.unimelb.edu.au with attachment c:\Users\Kunal Patel\D folder\MDS 

In [17]:
# mailerDDS2.send_emails()
print(mailerDDS2.no_email_list)
# ['Sadeghlou Armin (1524653)']

['Sadeghlou Armin (1524653)']


In [7]:
mailerDDS3GPPerio.send_emails()
print(mailerDDS3GPPerio.no_email_list)

Sending emails for cohort: DDS3, with size: (98, 6)
Sending email to ayomide.afolabi@student.unimelb.edu.au for Ayomide Afolabi (1184941), type: 1
Found report file for Afolabi Ayomide (1184941): Afolabi Ayomide (1184941).pdf
Sending email to ayomide.afolabi@student.unimelb.edu.au with attachment 2025\DDS3\GP+Perio\Student Reports GP+Perio\DDS3\Afolabi Ayomide (1184941).pdf
Email sent to ayomide.afolabi@student.unimelb.edu.au with attachment 2025\DDS3\GP+Perio\Student Reports GP+Perio\DDS3\Afolabi Ayomide (1184941).pdf

Sending email to emily.agnew@student.unimelb.edu.au for Emily Agnew (1402789), type: 1
Found report file for Agnew Emily (1402789): Agnew Emily (1402789).pdf
Sending email to emily.agnew@student.unimelb.edu.au with attachment c:\Users\Kunal Patel\D folder\MDS Work\2025\DDS3\GP+Perio\Student Reports GP+Perio\DDS3\Agnew Emily (1402789).pdf
Email sent to emily.agnew@student.unimelb.edu.au with attachment c:\Users\Kunal Patel\D folder\MDS Work\2025\DDS3\GP+Perio\Student Rep

In [9]:
mailerDDS3Specialist.send_emails()
print(mailerDDS3Specialist.no_email_list)

Sending emails for cohort: DDS3, with size: (98, 6)
Sending email to ayomide.afolabi@student.unimelb.edu.au for Ayomide Afolabi (1184941), type: 1
Found report file for Afolabi Ayomide (1184941): Afolabi  Ayomide  (1184941).pdf
Sending email to ayomide.afolabi@student.unimelb.edu.au with attachment 2025\DDS3\Specialist\Student Reports\DDS3 All\Afolabi  Ayomide  (1184941).pdf
Email sent to ayomide.afolabi@student.unimelb.edu.au with attachment 2025\DDS3\Specialist\Student Reports\DDS3 All\Afolabi  Ayomide  (1184941).pdf

Sending email to emily.agnew@student.unimelb.edu.au for Emily Agnew (1402789), type: 1
Found report file for Agnew Emily (1402789): Agnew  Emily  (1402789).pdf
Sending email to emily.agnew@student.unimelb.edu.au with attachment c:\Users\Kunal Patel\D folder\MDS Work\2025\DDS3\Specialist\Student Reports\DDS3 All\Agnew  Emily  (1402789).pdf
Email sent to emily.agnew@student.unimelb.edu.au with attachment c:\Users\Kunal Patel\D folder\MDS Work\2025\DDS3\Specialist\Student 

In [20]:
# mailerBOH2.send_emails()
print(mailerBOH2.no_email_list)

['Pham Linda (1088656)', 'Tan Natasha (1617738)']


Only Natasha Tan is not here

In [ ]:
import numpy as np
import pandas as pd

# Parameters
monthlyInvestment = 1000
annualRate = 0.33
monthlyRate = annualRate / 12
yearsList = [1, 2, 5, 10, 15, 20, 25, 30]

# Calculating future values
results = []
for years in yearsList:
    months = years * 12
    fv = monthlyInvestment * ((1 + monthlyRate)**months - 1) / monthlyRate
    results.append({"Years": years, "Future Value": fv})

# Creating DataFrame for clear representation
df = pd.DataFrame(results)
df['Future Value'] = df['Future Value'].apply(lambda x: f"${x:,.1f}")
# df['Future Value'] = df['Future Value'].str.replace('$', '').str.replace(',', '').astype(float)

display(df)
